In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm

In [2]:
file_path = '../../../data/BPI Challenge 2017.xes'
event_log = pm4py.read_xes(file_path)

parsing log, completed traces ::   0%|          | 0/31509 [00:00<?, ?it/s]

In [3]:
event_log

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202262,Deleted,User_1,W_Call after offers,Workflow,Workitem_1817549786,ate_abort,2017-01-06 06:33:02.212000+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202263,Created,User_1,W_Call after offers,Workflow,Workitem_363876066,schedule,2017-01-06 06:33:02.221000+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202264,statechange,User_28,A_Cancelled,Application,ApplState_1869071797,complete,2017-01-16 09:51:21.114000+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202265,statechange,User_28,O_Cancelled,Offer,OfferState_420066181,complete,2017-01-16 09:51:21.139000+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_1580299144


In [8]:
example_application = event_log[event_log['case:concept:name'] == 'Application_1283264254'].copy()

In [9]:
example_application

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
570711,Created,User_1,A_Create Application,Application,Application_1283264254,complete,2016-07-04 09:34:53.112000+00:00,"Other, see explanation",New credit,Application_1283264254,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570712,statechange,User_1,A_Submitted,Application,ApplState_1072502574,complete,2016-07-04 09:34:53.237000+00:00,"Other, see explanation",New credit,Application_1283264254,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570713,Created,User_1,W_Handle leads,Workflow,Workitem_137879142,schedule,2016-07-04 09:34:53.636000+00:00,"Other, see explanation",New credit,Application_1283264254,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570714,Obtained,User_38,W_Handle leads,Workflow,Workitem_1104763209,start,2016-07-04 10:20:10.415000+00:00,"Other, see explanation",New credit,Application_1283264254,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570715,Deleted,User_38,W_Handle leads,Workflow,Workitem_420687419,complete,2016-07-04 10:21:41.308000+00:00,"Other, see explanation",New credit,Application_1283264254,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570787,Obtained,User_2,W_Call incomplete files,Workflow,Workitem_183031422,resume,2016-12-19 19:00:29.725000+00:00,"Other, see explanation",New credit,Application_1283264254,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570788,Released,User_2,W_Call incomplete files,Workflow,Workitem_910228405,suspend,2016-12-19 19:01:59.720000+00:00,"Other, see explanation",New credit,Application_1283264254,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570789,statechange,User_5,A_Cancelled,Application,ApplState_1176688132,complete,2016-12-20 11:19:12.912000+00:00,"Other, see explanation",New credit,Application_1283264254,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
570790,statechange,User_5,O_Cancelled,Offer,OfferState_714450772,complete,2016-12-20 11:19:12.933000+00:00,"Other, see explanation",New credit,Application_1283264254,5000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_137022571


In [16]:
example_application['new_concept'] = example_application['concept:name'] + '__' + example_application['lifecycle:transition'] + '_' + example_application['EventID']
example_application = example_application[example_application['concept:name'].str.startswith('W_')]
#test_log = example_application[(test_log['lifecycle:transition'] == 'start') | (test_log['lifecycle:transition'] == 'complete')]
net, im, fm = pm4py.discover_petri_net_inductive(example_application, activity_key='new_concept', case_id_key='case:concept:name', timestamp_key='time:timestamp')
#pm4py.view_petri_net(net, im, fm)
pm4py.save_vis_petri_net(net, im, fm, 'test.svg')

/tmp/ipykernel_726603/4111699374.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  example_application['new_concept'] = example_application['concept:name'] + '__' + example_application['lifecycle:transition'] + '_' + example_application['EventID']


''